In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *
import pandas as pd
import numpy as np
from itertools import cycle

In [ ]:
#| export
def foo(): pass

In [ ]:
#|export
def say_hello(to):
    "Say hello to somebody"
    return f'Hello {to}!'

In [ ]:
say_hello("water")

'Hello water!'

## test index

In [ ]:
from IPython.display import display,SVG

In [ ]:
display(SVG('<svg height="100" xmlns="http://www.w3.org/2000/svg"><circle cx="50" cy="50" r="40"/></svg>'))

In [ ]:
assert say_hello("Hamel")=="Hello Hamel!"

In [ ]:
say_hello("Isaac")

'Hello Isaac!'

## test dataframe


In [ ]:
#|export

def generate_fake_dataframe(size, cols, col_names = None, intervals = None, seed = None):
    'generate long fake dataframe'
    
    categories_dict = {'animals': ['cow', 'rabbit', 'duck', 'shrimp', 'pig', 'goat', 'crab', 'deer', 'bee', 'sheep', 'fish', 'turkey', 'dove', 'chicken', 'horse'],
                       'names'  : ['James', 'Mary', 'Robert', 'Patricia', 'John', 'Jennifer', 'Michael', 'Linda', 'William', 'Elizabeth', 'Ahmed', 'Barbara', 'Richard', 'Susan', 'Salomon', 'Juan Luis'],
                       'cities' : ['Stockholm', 'Denver', 'Moscow', 'Marseille', 'Palermo', 'Tokyo', 'Lisbon', 'Oslo', 'Nairobi', 'Río de Janeiro', 'Berlin', 'Bogotá', 'Manila', 'Madrid', 'Milwaukee'],
                       'colors' : ['red', 'orange', 'yellow', 'green', 'blue', 'indigo', 'purple', 'pink', 'silver', 'gold', 'beige', 'brown', 'grey', 'black', 'white']
                      }
    default_intervals = {"i" : (0,10), "f" : (0,100), "c" : ("names", 5), "d" : ("2020-01-01","2020-12-31")}
    rng = np.random.default_rng(seed)

    first_c = default_intervals["c"][0]
    categories_names = cycle([first_c] + [c for c in categories_dict.keys() if c != first_c])
    default_intervals["c"] = (categories_names, default_intervals["c"][1])
    
    if isinstance(col_names,list):
        assert len(col_names) == len(cols), f"The fake DataFrame should have {len(cols)} columns but col_names is a list with {len(col_names)} elements"
    elif col_names is None:
        suffix = {"c" : "cat", "i" : "int", "f" : "float", "d" : "date"}
        col_names = [f"column_{str(i)}_{suffix.get(col)}" for i, col in enumerate(cols)]

    if isinstance(intervals,list):
        assert len(intervals) == len(cols), f"The fake DataFrame should have {len(cols)} columns but intervals is a list with {len(intervals)} elements"
    else:
        if isinstance(intervals,dict):
            assert len(set(intervals.keys()) - set(default_intervals.keys())) == 0, f"The intervals parameter has invalid keys"
            default_intervals.update(intervals)
        intervals = [default_intervals[col] for col in cols]
    df = pd.DataFrame()
    for col, col_name, interval in zip(cols, col_names, intervals):
        if interval is None:
            interval = default_intervals[col]
        assert (len(interval) == 2 and isinstance(interval, tuple)) or isinstance(interval, list), f"This interval {interval} is neither a tuple of two elements nor a list of strings."
        if col in ("i","f","d"):
            start, end = interval
        if col == "i":
            df[col_name] = rng.integers(start, end, size)
        elif col == "f":
            df[col_name] = rng.uniform(start, end, size)
        elif col == "c":
            if isinstance(interval, list):
                categories = np.array(interval)
            else:
                cat_family, length = interval
                if isinstance(cat_family, cycle):
                    cat_family = next(cat_family)
                assert cat_family in categories_dict.keys(), f"There are no samples for category '{cat_family}'. Consider passing a list of samples or use one of the available categories: {categories_dict.keys()}"
                categories = rng.choice(categories_dict[cat_family], length, replace = False, shuffle = True)
            df[col_name] = rng.choice(categories, size, shuffle = True)
        elif col == "d":
            df[col_name] = rng.choice(pd.date_range(start, end), size)
    return df 

In [ ]:
generate_fake_dataframe(size = 1000, cols =  "cififficcdccc")

,column_0_cat,column_1_int,column_2_float,column_3_int,column_4_float,column_5_float,column_6_int,column_7_cat,column_8_cat,column_9_date,column_10_cat,column_11_cat,column_12_cat
0,Barbara,2,88.771278,2,19.689053,23.091290,3,goat,Nairobi,2020-07-15,white,Ahmed,goat
1,Barbara,2,80.510164,2,12.238819,88.086135,5,fish,Río de Janeiro,2020-07-30,yellow,William,goat
2,John,9,96.866835,7,29.996597,33.499301,6,deer,Río de Janeiro,2020-10-16,purple,James,sheep
3,Salomon,5,97.931690,3,85.637934,92.795474,5,fish,Nairobi,2020-01-06,yellow,Salomon,sheep
4,James,7,77.369389,8,51.328403,36.575868,2,turkey,Río de Janeiro,2020-07-19,purple,Mary,goat
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Salomon,4,96.162028,7,72.745658,94.497304,0,deer,Río de Janeiro,2020-11-23,yellow,Ahmed,goat
996,Robert,5,44.222819,3,1.979279,95.665325,4,fish,Milwaukee,2020-06-22,white,James,sheep
997,Robert,4,51.391188,2,11.194260,73.026377,4,fish,Milwaukee,2020-01-12,purple,James,chicken
998,Barbara,6,85.509794,6,38.235776,9.154935,0,bee,Río de Janeiro,2020-03-10,grey,Salomon,sheep


testing text with long dataframes

In [ ]:
generate_fake_dataframe(size = 1000, cols =  "cififficcdccdddcffic")

,column_0_cat,column_1_int,column_2_float,column_3_int,column_4_float,column_5_float,column_6_int,column_7_cat,column_8_cat,column_9_date,column_10_cat,column_11_cat,column_12_date,column_13_date,column_14_date,column_15_cat,column_16_float,column_17_float,column_18_int,column_19_cat
0,Salomon,4,58.261916,3,82.304096,62.538647,7,fish,Río de Janeiro,2020-06-15,black,Richard,2020-04-07,2020-01-16,2020-05-22,dove,91.046081,45.628899,7,Berlin
1,Jennifer,9,1.120511,1,14.164167,41.253572,4,duck,Palermo,2020-07-31,black,Robert,2020-10-16,2020-11-15,2020-04-22,pig,90.189140,82.729162,2,Bogotá
2,Jennifer,9,94.617821,0,91.508622,43.173473,9,sheep,Bogotá,2020-01-28,red,Richard,2020-06-19,2020-10-31,2020-04-30,dove,43.017002,99.820368,0,Berlin
3,Salomon,9,73.093203,2,88.149547,40.680560,7,sheep,Palermo,2020-06-20,red,Richard,2020-03-05,2020-04-04,2020-06-09,bee,92.414576,15.230226,7,Río de Janeiro
4,Michael,8,25.583190,8,23.221183,45.528823,2,pig,Río de Janeiro,2020-04-30,black,Elizabeth,2020-05-25,2020-03-07,2020-05-27,turkey,31.456601,28.095314,8,Manila
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,James,7,24.476542,0,85.528245,23.941403,9,duck,Bogotá,2020-04-23,yellow,Elizabeth,2020-12-24,2020-02-22,2020-11-07,pig,92.477320,72.269486,6,Bogotá
996,Michael,7,52.334108,7,45.453670,63.862723,9,pig,Tokyo,2020-01-20,red,Salomon,2020-01-30,2020-06-08,2020-06-12,dove,20.134985,47.025609,8,Berlin
997,Robert,7,10.190453,3,6.677282,47.725178,4,fish,Bogotá,2020-08-22,red,Robert,2020-11-18,2020-04-10,2020-11-15,bee,47.283578,74.731077,3,Bogotá
998,Salomon,9,32.450759,4,66.172942,2.640716,8,duck,Tokyo,2020-12-12,red,Robert,2020-06-27,2020-10-05,2020-06-20,pig,42.745923,21.517031,0,Manila


In [ ]:
#| hide
import nbdev
nbdev.nbdev_export()